In [32]:
import pandas as pd
import json
import os
from sqlalchemy import create_engine
import pymysql
import yaml
from datetime import datetime, timedelta
import boto3
import numpy as np
from unidecode import unidecode



pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [33]:
# Function to read YAML file
def read_yaml(file_path):
    with open(file_path, 'r') as file:
        try:
            # Load the YAML content into a Python dictionary
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as exc:
            print(f"Error reading YAML file: {exc}")
            return None

In [34]:
# Example date in datetime format
start_date = datetime(2024, 8, 9) # year, month, day

date_plus_one_week = start_date + timedelta(weeks=1)
end_date_naming = start_date + timedelta(days=6)

start_date = start_date.strftime("%Y-%m-%d")
end_date = date_plus_one_week.strftime("%Y-%m-%d")
end_date_naming = end_date_naming.strftime("%Y-%m-%d")


print(start_date)
print(end_date)
print(end_date_naming)

2024-08-09
2024-08-16
2024-08-15


In [35]:
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
extracting_dir = os.path.join(parent_dir, 'extracted_data/')

In [36]:
#### Extracting MYSQL configuration files

file_path = os.path.join(parent_dir,'access_keys.yaml')
config = read_yaml(file_path)

access_key = config.get('AWS_creds')['access_key']
secret_access_key = config.get('AWS_creds')['secret_access_key']


#### Area table parsing

In [37]:
file_name = os.path.join(extracting_dir,'area_data.json')


with open(file_name, 'r') as f:
    data = json.load(f)['areas']

#print(data)

#fitting json into pandas_dataframe

area_df = pd.DataFrame(data)
area_df.head()

,id,name,countryCode,flag,parentAreaId,parentArea
0,2000,Afghanistan,AFG,None,2014.0,Asia
1,2001,Africa,AFR,None,2267.0,World
2,2002,Albania,ALB,None,2077.0,Europe
3,2004,Algeria,ALG,None,2001.0,Africa
4,2005,American Samoa,ASM,None,2175.0,Oceania


#### Competitions table

In [38]:
file_name = os.path.join(extracting_dir,'comp_data.json')


with open(file_name, 'r') as f:
    data = json.load(f)['competitions']


comp_df = pd.DataFrame(data)
#parsing only area id from the area column and leaving out the other information

# comp_df['area_id'] = comp_df['area'].apply(lambda x: x['id'])
# comp_df['country'] = comp_df['area'].apply(lambda x: x['name'])

# comp_df.drop(columns = ['area'],inplace = True)
comp_df = comp_df.drop(columns = ['currentSeason'])

##normalizing code:
comp_df_filtered = pd.json_normalize(data,sep = '_')

comp_df_filtered = comp_df_filtered.loc[:, ~comp_df_filtered.columns.str.startswith('currentSeason')]



#selecting relvant columns only

rel_cols = ['id', 'name', 'code','type','area_id','area_name']

comp_df_filtered = comp_df_filtered[rel_cols]

comp_df_filtered.head(5)

,id,name,code,type,area_id,area_name
0,2013,Campeonato Brasileiro Série A,BSA,LEAGUE,2032,Brazil
1,2016,Championship,ELC,LEAGUE,2072,England
2,2021,Premier League,PL,LEAGUE,2072,England
3,2001,UEFA Champions League,CL,CUP,2077,Europe
4,2018,European Championship,EC,CUP,2077,Europe


#### Note that to extract a sub_dictionary inside a dictionary, useapply and lambda functions to extract specific keys of the sub_dictionary

Example:comp_df['area_id'] = comp_df['area'].apply(lambda x: x['id'])

#### Teams table

In [39]:
file_name = os.path.join(extracting_dir,'teams_data.json')


with open(file_name, 'r') as f:
    data = json.load(f)['teams']


teams_df = pd.DataFrame(data)


rel_cols = ['id','name','shortName','tla']

teams_df = teams_df[rel_cols]



print(teams_df.head(5))

   id                 name   shortName  tla
0   1           1. FC Köln  1. FC Köln  KOE
1   2  TSG 1899 Hoffenheim  Hoffenheim  TSG
2   3  Bayer 04 Leverkusen  Leverkusen  B04
3   4    Borussia Dortmund    Dortmund  BVB
4   5    FC Bayern München      Bayern  FCB


#### Finally Parsing Matches table

In [40]:
file_name = os.path.join(extracting_dir,f'match_data{start_date}to{end_date_naming}.json')

with open(file_name, 'r') as f:
    data = json.load(f)['matches']

matches_df = pd.json_normalize(data,max_level = 2,sep = '_')

matches_df['winner'] = matches_df.apply(lambda x: x['homeTeam_name'] if x['score_winner'] == 'HOME_TEAM' else (x['awayTeam_name'] if x['score_winner'] else 'DRAW'), axis = 1)
matches_df['Matchweek'] = 1

rel_cols = ['id','utcDate','Matchweek','stage','area_id','area_name','competition_id','homeTeam_id','awayTeam_id','score_winner','score_fullTime_home','score_fullTime_away']
matches_df = matches_df[rel_cols]

matches_df['utcDate'] = pd.to_datetime(matches_df['utcDate']).dt.date

matches_df['Matchweek'] = 1

matches_df.head(2)

,id,utcDate,Matchweek,stage,area_id,area_name,competition_id,homeTeam_id,awayTeam_id,score_winner,score_fullTime_home,score_fullTime_away
0,498993,2024-08-09,1,REGULAR_SEASON,2163,Netherlands,2003,677,681,HOME_TEAM,4,1
1,500445,2024-08-09,1,REGULAR_SEASON,2072,England,2016,59,342,HOME_TEAM,4,2


In [42]:
matches_df.tail()

,id,utcDate,Matchweek,stage,area_id,area_name,competition_id,homeTeam_id,awayTeam_id,score_winner,score_fullTime_home,score_fullTime_away
45,517539,2024-08-15,1,LAST_16,2220,South America,2152,1770,1769,HOME_TEAM,2,1
46,517536,2024-08-15,1,LAST_16,2220,South America,2152,2073,6667,AWAY_TEAM,0,1
47,498613,2024-08-15,1,REGULAR_SEASON,2224,Spain,2014,77,82,DRAW,1,1
48,498614,2024-08-15,1,REGULAR_SEASON,2224,Spain,2014,90,298,DRAW,1,1
49,517538,2024-08-15,1,LAST_16,2220,South America,2152,7055,1776,DRAW,0,0


#### You see that some of the above dataframes have a subschema (sub_dictionaries) inside them. There is a need to 'flatten' them before we load this data into MySQL database. This process is called "Normalization"

#### Final data frames

In [43]:
matches_df.head()

,id,utcDate,Matchweek,stage,area_id,area_name,competition_id,homeTeam_id,awayTeam_id,score_winner,score_fullTime_home,score_fullTime_away
0,498993,2024-08-09,1,REGULAR_SEASON,2163,Netherlands,2003,677,681,HOME_TEAM,4,1
1,500445,2024-08-09,1,REGULAR_SEASON,2072,England,2016,59,342,HOME_TEAM,4,2
2,500446,2024-08-09,1,REGULAR_SEASON,2072,England,2016,1081,356,AWAY_TEAM,0,2
3,504020,2024-08-09,1,REGULAR_SEASON,2187,Portugal,2017,498,496,HOME_TEAM,3,1
4,500451,2024-08-10,1,REGULAR_SEASON,2072,England,2016,384,346,AWAY_TEAM,2,3


In [44]:
comp_df_filtered.head()

,id,name,code,type,area_id,area_name
0,2013,Campeonato Brasileiro Série A,BSA,LEAGUE,2032,Brazil
1,2016,Championship,ELC,LEAGUE,2072,England
2,2021,Premier League,PL,LEAGUE,2072,England
3,2001,UEFA Champions League,CL,CUP,2077,Europe
4,2018,European Championship,EC,CUP,2077,Europe


In [45]:
teams_df.head()

,id,name,shortName,tla
0,1,1. FC Köln,1. FC Köln,KOE
1,2,TSG 1899 Hoffenheim,Hoffenheim,TSG
2,3,Bayer 04 Leverkusen,Leverkusen,B04
3,4,Borussia Dortmund,Dortmund,BVB
4,5,FC Bayern München,Bayern,FCB


In [46]:
area_df.head()

,id,name,countryCode,flag,parentAreaId,parentArea
0,2000,Afghanistan,AFG,None,2014.0,Asia
1,2001,Africa,AFR,None,2267.0,World
2,2002,Albania,ALB,None,2077.0,Europe
3,2004,Algeria,ALG,None,2001.0,Africa
4,2005,American Samoa,ASM,None,2175.0,Oceania


#### Loading tables into MYSQL server

In [47]:
# user = 'root'

# file_name = os.path.join(parent_dir,'mysql_password.txt')

# with open(file_name, 'r') as f:
#     password = f.read()
# host = 'localhost'
# port = 3306
# database = 'football_analytics'

# # Create the SQLAlchemy engine with host and port
# engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

In [48]:
# engine

In [49]:
# ##dataframes to be loaded into MySQL

# comp_df_filtered.to_sql('competition',con = engine, if_exists = 'replace',index = False)

# teams_df.to_sql('teams',con = engine, if_exists = 'replace',index = False)

# area_df.to_sql('area',con = engine, if_exists = 'replace',index = False)

# matches_df.to_sql('matches',con = engine, if_exists = 'replace',index = False)

In [50]:
## some minorchanges

## REMOVE AREA NAME FROM matches
## remove flag from area
## convert parentAreaID to int

## remove parent area name (self referential table)

comp_df_filtered = comp_df_filtered.drop(columns = 'area_name')

def apply_unidecode(x):

    if pd.isna(x):
        return x
    return unidecode(x)


matches_df = matches_df.drop(columns='area_name')
area_df = area_df.drop(columns = ['flag','parentArea'])

teams_df['name'] = teams_df['name'].apply(apply_unidecode)
teams_df['shortName'] = teams_df['shortName'].apply(apply_unidecode)

def conv_flt_int(x):

    if pd.isna(x):
        return x

    return int(x)


area_df['parentAreaId'] = area_df['parentAreaId'].apply(conv_flt_int)
area_df['parentAreaId'] = area_df['parentAreaId'].astype('Int64')

In [51]:
## Saving dataframes in csv

# parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
csv_data_dir = os.path.join(parent_dir, 'CSV_data')
matches_df.to_csv(os.path.join(csv_data_dir, 'matches.csv'), index=False)
comp_df_filtered.to_csv(os.path.join(csv_data_dir, 'competition.csv'), index=False)
area_df.to_csv(os.path.join(csv_data_dir, 'area.csv'), index=False)
teams_df.to_csv(os.path.join(csv_data_dir, 'teams.csv'), index=False, encoding='utf-8')

In [52]:
### Loading files to s3 and then later loading tables in AWS Redshift

import boto3

# Initialize the S3 client
s3_client = boto3.client('s3',
                aws_access_key_id=access_key,
                aws_secret_access_key=secret_access_key,
                region_name='us-east-1')

# Specify your S3 bucket name
BUCKET_NAME = 'football-analytics-amark'

# Define local file paths and corresponding S3 keys
files = {
    os.path.join(csv_data_dir, 'matches.csv'): 'matches/matches.csv',
    os.path.join(csv_data_dir, 'competition.csv'): 'competition/competition.csv',
    os.path.join(csv_data_dir, 'area.csv'): 'area/area.csv',
    os.path.join(csv_data_dir, 'teams.csv'): 'teams/teams.csv'
}

# Upload each file to S3
for local_path, s3_key in files.items():
    try:
        s3_client.upload_file(local_path, BUCKET_NAME, s3_key)
        print(f"Uploaded {local_path} to s3://{BUCKET_NAME}/{s3_key}")
    except Exception as e:
        print(f"Error uploading {local_path}: {e}")


Uploaded c:\Users\pragn\OneDrive\Documents\football_analytics_git\football_analytics\CSV_data\matches.csv to s3://football-analytics-amark/matches/matches.csv
Uploaded c:\Users\pragn\OneDrive\Documents\football_analytics_git\football_analytics\CSV_data\competition.csv to s3://football-analytics-amark/competition/competition.csv
Uploaded c:\Users\pragn\OneDrive\Documents\football_analytics_git\football_analytics\CSV_data\area.csv to s3://football-analytics-amark/area/area.csv
Uploaded c:\Users\pragn\OneDrive\Documents\football_analytics_git\football_analytics\CSV_data\teams.csv to s3://football-analytics-amark/teams/teams.csv
